<a href="https://colab.research.google.com/github/JBenedictM/501-A4-TF/blob/master/501_A4_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Upload notMNIST.npz

In [0]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/notMNIST.npz"

In [0]:
import tensorflow as tf
import numpy as np

print("--Get data--")
# path is defined by the cell above
with np.load(path, allow_pickle=True) as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

print("--Process data--")
print(len(y_train))
x_train, x_test = x_train / 255.0, x_test / 255.0

#shape = (batch_size, height, width, depth)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
 
 # new model employes more convolution layers 
 # each layer also contains more kernels
print("--Make model--")
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, input_shape=(28, 28, 1), kernel_size=(3, 3), activation="relu", padding="same"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"),
  tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding="same"),
  tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"),
  tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("--Fit model--")
val_loss_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=20)
checkpoint_val = tf.keras.callbacks.ModelCheckpoint("best_notmnist_model.h5", monitor="val_loss", mode="min", save_best_only=True, verbose=1)
checkpoint_train = tf.keras.callbacks.ModelCheckpoint("best_notmnist_model.h5", monitor="loss", mode="min", save_best_only=True, verbose=1)

cb = [val_loss_callback, checkpoint_val, checkpoint_train]

model.fit(x_train, y_train, epochs=99999999,  verbose=1, validation_data=(x_test, y_test), callbacks=cb)

print("--Evaluate model--")
model_loss, model_acc = model.evaluate(x_test,  y_test, verbose=2)
model.summary()
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")

In [0]:
#Save Model to notMNIST.h5

In [0]:
from google.colab import files
files.download('best_notmnist_model.h5')